# Grasp Planning

🚧 This notebook is still under construction.

> ℹ️ If you want to run this notebook, you will need to install additional dependencies.
> You can try creating the conda environment in `environment-dev.yaml`, which should include the necessary dependencies.

In [ ]:
import os
from pathlib import Path


current_team = "dev_team"
dataset_dir = Path(f"data/remote_dry_run_2024-04-26/{current_team}")

sample_id = "2024-04-23_10-20-07-968516"

sample_dir = dataset_dir / f"sample_{sample_id}"
grasps_dir = dataset_dir / f"grasps_{sample_id}" # TODO throughout this notebook add grasp.json files 

os.path.exists(sample_dir)

In [ ]:
from cloth_tools.dataset.format import load_competition_observation

observation_start_dir = sample_dir / "observation_start"

observation = load_competition_observation(observation_start_dir)

In [ ]:
from cloth_tools.drake.scenes import make_drake_scene_from_observation

scene = make_drake_scene_from_observation(observation, include_cloth_obstacle=False)
scene_with_cloth = make_drake_scene_from_observation(observation, include_cloth_obstacle=True)

In [ ]:
from functools import partial

from cloth_tools.kinematics.constants import TCP_TRANSFORM
from cloth_tools.kinematics.inverse_kinematics import inverse_kinematics_in_world_fn

X_W_LCB = observation.arm_left_pose_in_world
X_W_RCB = observation.arm_right_pose_in_world

inverse_kinematics_left_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_LCB, tcp_transform=TCP_TRANSFORM)
inverse_kinematics_right_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_RCB, tcp_transform=TCP_TRANSFORM)

In [ ]:
from cloth_tools.drake.scenes import make_dual_arm_collision_checker


collision_checker_no_cloth = make_dual_arm_collision_checker(scene)
collision_checker_with_cloth = make_dual_arm_collision_checker(scene_with_cloth)

In [ ]:
from cloth_tools.kinematics.constants import JOINT_BOUNDS
from airo_planner import DualArmOmplPlanner

planner_pregrasp = DualArmOmplPlanner(
    is_state_valid_fn=collision_checker_with_cloth.CheckConfigCollisionFree,
    inverse_kinematics_left_fn=inverse_kinematics_left_fn,
    inverse_kinematics_right_fn=inverse_kinematics_right_fn,
    joint_bounds_left=JOINT_BOUNDS,
    joint_bounds_right=JOINT_BOUNDS,
)

In [ ]:
from cloth_tools.annotation.grasp_annotation import grasp_hanging_cloth_pose
import numpy as np

grasp_pose_fixed = grasp_hanging_cloth_pose(np.array([0, 0, 0.5]), np.array([1, 0, 0]), 0.0)
grasp_pose_fixed

In [ ]:
from cloth_tools.planning.grasp_planning import plan_pregrasp_and_grasp_trajectory
from airo_drake import animate_dual_joint_trajectory

trajectory_pregrasp_and_grasp = plan_pregrasp_and_grasp_trajectory(
    planner_pregrasp,
    grasp_pose_fixed,
    observation.arm_left_joints,
    observation.arm_right_joints,
    inverse_kinematics_left_fn,
    inverse_kinematics_right_fn,
    collision_checker_no_cloth.CheckConfigCollisionFree,
    scene.robot_diagram.plant(),
    with_left=False
)

animate_dual_joint_trajectory(
    scene_with_cloth.meshcat,
    scene_with_cloth.robot_diagram,
    scene_with_cloth.arm_left_index,
    scene_with_cloth.arm_right_index,
    trajectory_pregrasp_and_grasp,
)

print(f"You can see the trajectory animation at: {scene_with_cloth.meshcat.web_url()}")

In [ ]:
from cloth_tools.planning.grasp_planning import ExhaustedOptionsError


def is_grasp_executable_fn(observation, grasp_pose) -> bool:
    scene = make_drake_scene_from_observation(observation, include_cloth_obstacle=False)
    scene_with_cloth = make_drake_scene_from_observation(observation, include_cloth_obstacle=True)

    X_W_LCB = observation.arm_left_pose_in_world
    X_W_RCB = observation.arm_right_pose_in_world

    inverse_kinematics_left_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_LCB, tcp_transform=TCP_TRANSFORM)
    inverse_kinematics_right_fn = partial(inverse_kinematics_in_world_fn, X_W_CB=X_W_RCB, tcp_transform=TCP_TRANSFORM)

    collision_checker_no_cloth = make_dual_arm_collision_checker(scene)
    collision_checker_with_cloth = make_dual_arm_collision_checker(scene_with_cloth)

    planner_pregrasp = DualArmOmplPlanner(
        is_state_valid_fn=collision_checker_with_cloth.CheckConfigCollisionFree,
        inverse_kinematics_left_fn=inverse_kinematics_left_fn,
        inverse_kinematics_right_fn=inverse_kinematics_right_fn,
        joint_bounds_left=JOINT_BOUNDS,
        joint_bounds_right=JOINT_BOUNDS,
    )

    try:
        plan_pregrasp_and_grasp_trajectory(
            planner_pregrasp,
            grasp_pose,
            observation.arm_left_joints,
            observation.arm_right_joints,
            inverse_kinematics_left_fn,
            inverse_kinematics_right_fn,
            collision_checker_no_cloth.CheckConfigCollisionFree,
            scene.robot_diagram.plant(),
            with_left=False,
        )
    except ExhaustedOptionsError:
        return False

    return True


is_grasp_executable_fn(observation, grasp_pose_fixed)

In [ ]:
grasp_pose_unreachable = grasp_hanging_cloth_pose(np.array([0, 0, 1.5]), np.array([1, 0, 0]), 0.5)
print(grasp_pose_unreachable)

is_grasp_executable_fn(observation, grasp_pose_unreachable)